In [1]:
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import torch
import pandas as pd
from tqdm import tqdm

In [ ]:
import cv2

def cxcywh2xyxy(x):
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[..., 0] = x[..., 0] - x[..., 2]/2  # x center
    y[..., 1] = x[..., 1] - x[..., 3]/2  # y center
    y[..., 2] = x[..., 0] + x[..., 2]/2  # width
    y[..., 3] = x[..., 1] + x[..., 3]/2  # height
    return y

In [ ]:
imagesPath = "*RPC/test/images/*.png"
labelsPath = "*RPC/test/labels/*.txt"
SupportPath = "RPC/support/images/"
best = 'runs/oneshot/yolov8n_RPC/weights/best.pt'

# imagesPath = "*COCO1/valid/images/*.png"
# labelsPath = "*COCO1/valid/labels/*.txt"
# SupportPath = "COCO1/support/images/"
# best = 'runs/oneshot/yolov8n_COCO1__support0/weights/best.pt'

In [ ]:
import glob
filesName =  glob.glob(imagesPath)
filesName.sort()
labelsName = glob.glob(labelsPath)
labelsName.sort()
print(len(filesName))
print(len(labelsName))

In [ ]:
from ultralytics import YOLO
from PIL import Image

model = YOLO(best)  # load a pretrained YOLOv8n model

In [ ]:
target = []
predictions = []
  
for f,l in tqdm(zip(filesName,labelsName), total=len(filesName)):
    assert f.split('/')[-1][:-4] == l.split('/')[-1][:-4] 

    image = cv2.imread(f)

    data = pd.read_csv(l, header = None, delimiter=' ')
    cls = data[0].unique()

    pred_boxes = []
    pred_scores = []
    pred_labels = []

    true_boxes = []
    true_labels = []

    for c in cls:

        support = cv2.imread(SupportPath+str(c)+'.png')

        results = model.predict(source=image, support=support, verbose=False)  # predict on an image

        bboxes = results[0].boxes.data.cpu().numpy()

        if len(bboxes) != 0:
            bboxes[:,5] = c
            if bboxes.ndim == 1:
                bboxes = np.array([bboxes])
            bboxes[:,:4] = bboxes[:,:4]/results[0].orig_img.shape[0]
            for b,s,l in zip(bboxes[:,:4],bboxes[:,4],bboxes[:,5]):
                pred_boxes.append(b)
                pred_scores.append(s)
                pred_labels.append(l)


        label = data.to_numpy()[:,:5]
        label = label[np.where(label[:,0] == c)]
        label[:,1:5] = cxcywh2xyxy(label[:,1:5])

        for b,l in zip(label[:,1:5],label[:,0]):
            true_boxes.append(b)
            true_labels.append(l)

    true_boxes = np.array(true_boxes)
    true_labels = np.array(true_labels)
    pred_boxes = np.array(pred_boxes)
    pred_scores = np.array(pred_scores)
    pred_labels = np.array(pred_labels)

    predictions.append(
        dict(
            boxes=torch.as_tensor(pred_boxes),
            scores=torch.as_tensor(pred_scores),
            labels=torch.as_tensor(pred_labels, dtype=torch.int32),
            )
        )       
    
    target.append(
        dict(
            boxes=torch.as_tensor(true_boxes),
            labels=torch.as_tensor(true_labels, dtype=torch.int32).reshape(-1),
            )
        )       

In [ ]:
def plot(t,p, image):
    img = np.copy(image)
    # Blue color in BGR
    color = (255, 0, 0)
  
    # Line thickness of 2 px
    thickness = 2   
    for x1,y1,x2,y2 in t['boxes']*640:
        cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), color, thickness)

    # Green color in BGR
    color = (0, 255, 0)
  
    # Line thickness of 2 px
    thickness = 2   
    for x1,y1,x2,y2 in p['boxes']*640:
        cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), color, thickness)

    plt.figure()
    plt.imshow(img)
    plt.show()


In [ ]:
index = 7

f = filesName[index]
image = Image.open(f)
plot(target[index],predictions[index],image)

In [ ]:
from torch import tensor
from torchmetrics.detection.mean_ap import MeanAveragePrecision
preds = predictions
target = target
metric = MeanAveragePrecision()
metric.update(preds, target)
print(metric.compute())
# fig_, ax_ = metric.plot()

In [ ]:
# from torch import tensor
# from torchmetrics.detection.mean_ap import MeanAveragePrecision
# preds = [dict(
#      boxes=tensor([[0.0, 0.0, 100.0, 100.0]]),
#     scores=tensor([0.8]),
#      labels=tensor([0]),
#  ),dict(
#      boxes=tensor([[0.0, 0.0, 100.0, 100.0]]),
#     scores=tensor([0.8]),
#      labels=tensor([0]),
#  )]
# target = [dict(
#      boxes=tensor([[0.0, 0.0, 110.0, 110.0]]),
#      labels=tensor([0]),
#  ),dict(
#      boxes=tensor([[0.0, 0.0, 110.0, 110.0]]),
#      labels=tensor([0]),
#  )]

# print(preds)

# print(target)

In [ ]:
# metric = MeanAveragePrecision()
# metric.update(preds, target)
# print(metric.compute())
# fig_, ax_ = metric.plot()
